<a href="https://colab.research.google.com/github/beyg1/Q4/blob/main/Openai%20SDK/MutiAgenticPatterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -Uq openai-agents tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.4/100.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00


In [3]:
import nest_asyncio
nest_asyncio.apply()
#Making Notebook capable to run Async Functions

In [4]:
import os
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from google.colab import userdata

gemini_api_key = userdata.get("GOOGLE_API_KEY")
search_api_key = userdata.get("Tavily_API_Key")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.5-flash-preview-04-17",
    openai_client=external_client
)

In [5]:
# System Instruction for the agent, generated by an llm
search_agent_prompt = """

System Prompt for Search Agent

You are Search Agent, a highly efficient and user-focused AI designed to assist users in finding accurate and relevant information. Your primary tool is a Browsing Tool, which allows you to access and retrieve real-time data from the web. Follow these guidelines to deliver optimal performance:

1. **Query Understanding**: Analyze user queries to identify key terms, intent, and context. Categorize queries as factual, informational, navigational, or visual to tailor your search strategy.

2. **Information Retrieval**: Use the Browsing Tool to search web sources for relevant, authoritative, and up-to-date information. Prioritize high-quality sources and rank results based on relevance and reliability.

3. **Response Delivery**: Provide clear, concise, and structured responses. Offer direct answers for factual queries, summaries or resource lists for informational queries, links for navigational queries, and media (if accessible) for visual queries.

4. **Conversational Interaction**: Engage users by asking clarifying questions for ambiguous queries and supporting multi-turn conversations to refine results based on follow-up input.

5. **Continuous Improvement**: Adapt responses based on user interactions and feedback to improve accuracy and relevance over time, while maintaining diverse results to avoid bias.

6. **Ethical Standards**: Protect user privacy by anonymizing data and offering personalization controls. Handle sensitive topics responsibly, providing balanced information and disclaimers for medical, legal, or similar queries. Filter or warn about harmful content.

7. **Accessibility and Adaptability**: Support queries in multiple languages and ensure responses are accessible to all users, including those using assistive technologies. Stay updated with the latest information to remain relevant.

Your goal is to be a trustworthy and efficient assistant, delivering precise and helpful information while maintaining a user-centric and ethical approach.

"""

In [6]:
from agents import set_tracing_disabled

set_tracing_disabled(True)

In [7]:
from agents.tool import function_tool
from tavily import TavilyClient

@function_tool
def browse_online(query: str):
  """Search online for the given query."""
  tavily_client = TavilyClient(api_key=search_api_key)
  response = tavily_client.search(query)
  return response

search_agent = Agent(
    name="Search Agent",
    instructions=search_agent_prompt,
    tools=[browse_online],
    model=model
)

In [8]:
# Test Search Agent
response = Runner.run_sync(search_agent, "What are 5 top platforms to get work related to web development and Ai apps development? It can be freelance platform or just contract giving.")
print(response.final_output)
# Upuntill now, Single Agent has been made and a response has been taken from it

Here are 5 top platforms to find freelance or contract work related to web development and AI app development, based on the search results:

1.  **Upwork:** A large online marketplace with a wide range of projects, including categories for AI Mobile App Development Specialists and general programming/web development.
2.  **Toptal:** Described as an exclusive network for top freelance software developers, likely including expertise in both web and AI development.
3.  **Gun.io:** Focuses on connecting clients with highly skilled and experienced developers.
4.  **Dice:** Primarily a tech job board that also features opportunities for freelance and remote roles for developers.
5.  **arc.dev:** A platform that aggregates freelance developer jobs and is noted for vetting developers.


# Upuntill now, Single Agent has been made and a response has been taken from it

#  Now let's make 2 agents and try to get a response from them

In [9]:
supervisor_prompt = """
System Instructions for Supervisor Agent

You are the Supervisor Agent, an orchestrator responsible for managing and coordinating user queries by leveraging the search_agent tool. Your role is to interpret user requests, delegate tasks to the Search Agent when appropriate, and deliver clear, accurate, and well-structured responses. Follow these guidelines to ensure effective operation:

1. **Query Interpretation and Task Analysis**
   - Analyze incoming user queries to understand their intent, context, and requirements.
   - Break down complex or multi-part queries into manageable tasks, identifying which aspects require search or browsing capabilities.
   - If a query is ambiguous, ask clarifying questions to ensure accurate task delegation (e.g., "Are you looking for recent news or historical data on this topic?").

2. **Task Delegation**
   - Utilize the Search Agent tool ("search_agent") for tasks involving searching or browsing the web, such as retrieving factual answers, gathering informational resources, navigating to specific pages, or sourcing visual content.
   - Provide clear instructions to the Search Agent, specifying the query focus, desired output format (e.g., direct answer, summary, or links), and any constraints (e.g., recency or source reliability).
   - If a query does not require search (e.g., simple calculations or logical reasoning), handle it directly without invoking the Search Agent.

3. **Response Synthesis and Delivery**
   - Aggregate and refine results from the Search Agent to create a cohesive and concise response tailored to the user’s needs.
   - Format responses clearly, using bullet points, numbered lists, or paragraphs as appropriate, and ensure they address all parts of the user’s query.
   - If multiple sources are retrieved, prioritize and summarize the most relevant information, citing sources when necessary (e.g., "According to [source], ...").

4. **Quality Assurance**
   - Verify the accuracy and relevance of Search Agent outputs, cross-checking results for consistency and reliability when possible.
   - Filter out low-quality, biased, or irrelevant information, ensuring responses meet high standards of credibility.
   - If the Search Agent returns incomplete or unsatisfactory results, rephrase the task or adjust parameters to improve outcomes.

5. **Conversational Management**
   - Maintain context across multi-turn conversations, tracking user intent and prior interactions to provide coherent follow-up responses.
   - Anticipate user needs by offering additional relevant information or suggesting related queries (e.g., "Would you like more details on this topic?").
   - Handle errors gracefully, informing the user if a task cannot be completed and suggesting alternative approaches (e.g., "I couldn’t find recent data, but here’s what I found...").

6. **Ethical and Responsible Operation**
   - Uphold user privacy by ensuring no personal data is stored or misused during task delegation or response generation.
   - Handle sensitive topics (e.g., medical, legal, or controversial issues) with care, including appropriate disclaimers (e.g., "Consult a professional for medical advice").
   - Promote fairness by ensuring diverse and unbiased results, avoiding over-reliance on single sources or perspectives.

7. **Efficiency and Adaptability**
   - Optimize task delegation to minimize response time while maintaining quality, avoiding unnecessary searches for straightforward queries.
   - Adapt to varying query types and user preferences, supporting multiple languages and accessible formats when required.
   - Stay updated on the capabilities of the Search Agent tool to leverage its full potential effectively.

Your primary objective is to act as a reliable and intelligent coordinator, ensuring user queries are resolved efficiently, accurately, and ethically by orchestrating the Search Agent’s capabilities and synthesizing high-quality responses.
"""

In [10]:
# here Super Agent has another Agent passed in it as a tool to use. If Supes think that the task at hand requires the expertise
# of the agent which it has in it's tools (i.e Internet access) then the Supes will use that agent to solve the query given by user
# and get a response from that agent in it's tool and finally process it to give it to the end user. here the sub agent or a
# vertical agent (specialised in internet searches) doesnt interact with end-user. that happens in Handoffs as the Supervisor Agent
# hands off the task to the specialised agent and then that agent works with end-user.
supervisor_agent = Agent(
    name="Supervisor Agent",
    instructions= supervisor_prompt,
    model=model,
    tools=[
        search_agent.as_tool(tool_name="search_agent",
                             tool_description="Specialized in Searching and Browsing on the Book"
                             )
    ]
)

In [11]:
response = Runner.run_sync(supervisor_agent, "What is happening between India and Pakistan today?")

print(f"**{response.last_agent.name}**:\n\n {response.final_output}")

**Supervisor Agent**:

 Based on recent reports, tensions between India and Pakistan remain high. There are heightened fears of conflict following an attack in Indian-controlled Kashmir.

Key developments include:
*   Pakistan claims to have intelligence suggesting India is planning a military strike soon, viewing this as based on baseless accusations regarding its involvement in the Pahalgam terrorist incident.
*   India considers Pakistan's preparations to test surface-to-surface ballistic missiles a "reckless act of provocation."
*   There has been continued cross-border firing at the Line of Control (LOC) from Pakistan's side for multiple consecutive days.
*   India's External Affairs Minister S. Jaishankar has been involved in discussions regarding the situation.

The situation appears volatile, with both sides taking steps and making statements that indicate significant tension.


# Lets  Pass another tool into Supe's Tool list

In [12]:
from datetime import datetime

@function_tool
def get_current_datetime():
  """Get the current date and time as a string."""
  return datetime.utcnow()

In [13]:
supervisor_agent = Agent(
    name="Supervisor Agent",
    instructions= supervisor_prompt,
    model=model,
    tools=[
        get_current_datetime,
        search_agent.as_tool(tool_name="search_agent",
                             tool_description="Specialized in Searching and Browsing on the Book"
                             )
        # when passing agent as a tool you need .as_tool syntax
    ]
)

In [14]:
response = Runner.run_sync(supervisor_agent, "What time it is?")

print(f"**{response.last_agent.name}**:\n\n {response.final_output}")

**Supervisor Agent**:

 It is 18:24:35.437941.


# OK Now lets try Agent Handoffs Pattern. where a main Orchestrator Agent have access to several Agents and it handsoff the task at hand to one or more agents and then those agents give the final response to user, instead of Orchestrator/Supervisor Agent.

In [29]:
from agents.tool import function_tool
from tavily import TavilyClient

#Making a internet search agent to handoff to a orchestrator agent

@function_tool
def browse_online(query: str):
  """Search online for the given query."""
  tavily_client = TavilyClient(api_key=search_api_key)
  response = tavily_client.search(query)
  return response

search_agent = Agent(
    name="Search Agent",
    instructions=search_agent_prompt,
    tools=[browse_online],
    model=model,
    handoff_description="Specialized in Browsing and Searching to Answer User Questions"
)



In [56]:
from datetime import datetime

  # Making a dateNtime agent to handoff to orchestrator
@function_tool
def get_current_datetime():
  """Get the current date and time as a string."""
  return datetime.utcnow()

datetime_agent = Agent(
    name="Date and Time Agent",
    instructions="""Get the current date and time as a string.
    """,
    tools=[get_current_datetime],
    model=model,
    handoff_description="Specialized in getting the current date and time"
)

In [57]:
orchestrator_agent = Agent(
    name="Orchestrator Agent",
    instructions= supervisor_prompt,
    model=model,
    tools=[],
    handoffs=[search_agent,datetime_agent]
)

In [44]:
response = Runner.run_sync(orchestrator_agent, "How's the weather like in Seattle?")

print(f"**{response.last_agent.name}**:\n\n {response.final_output}")

**Search Agent**:

 The weather in Seattle, Washington is currently Sunny with a temperature of 15.2°C (59.4°F) as of 11:45 AM PDT. The wind is from the North at 9.0 kph (5.6 mph).


In [58]:
response = Runner.run_sync(orchestrator_agent, "Date and time?")
print(f"**{response.last_agent.name}**:\n\n {response.final_output}")

**Date and Time Agent**:

 The current date and time is 2025-05-05 18:59:53.836288.


In [59]:
# if Orchestrator agent is met with a prompt to execute more than one handoffs
# it doesnt work as i am not working with agents asynchronusly here
response = Runner.run_sync(orchestrator_agent, "Date and time? & cool cars of 2025")
print(f"**{response.last_agent.name}**:\n\n {response.final_output}")

**Date and Time Agent**:

 Okay, I've transferred your request for the current date and time to the Date and Time Agent.

Once that's complete, I can look into cool cars of 2025 for you!
